Custom Vision

In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

In [2]:
ENDPOINT = "https://kmooccustomvision.cognitiveservices.azure.com/"
training_key = "3ad72efc976d4e7c92581146b6bd68c7"
prediction_key = "8cb537f042fe4ab081ab07ca14f33009"
prediction_resource_id = "/subscriptions/a4acd1e5-492e-4b21-b606-9da9d7ed3544/resourceGroups/KMOOC/providers/Microsoft.CognitiveServices/accounts/kmooccustomVision-Prediction"

In [3]:
publish_iteration_name = "classifyModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Create a new project
print ("Creating project...")
project_name = "My project"
#project = trainer.create_project(project_name)
project = trainer.get_project("c26f3045-a64d-4ccc-a44e-85ced931944a")

#$publish_iteration_name = "classifyModel"
#이거는 안되는거
#trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)
# trainer = CustomVisionTrainingClient(ENDPOINT, training_key)
# print ("Creating project...")
# project = trainer.create_project("My New Project2")

Creating project...


3개의 태그를 생성

In [4]:
Jajangmyeon_tag = trainer.create_tag(project.id, "Jajangmyeon")
Champon_tag = trainer.create_tag(project.id, "Champon")
Tangsuyug_tag = trainer.create_tag(project.id, "Tangsuyug")

CustomVisionErrorException: Name not unique: Jajangmyeon, Project: c26f3045-a64d-4ccc-a44e-85ced931944a

학습 시키는 방법

In [ ]:
import time
print("training...")
iteration = trainer.train_project(project.id)

while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(1)

trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

In [ ]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [ ]:
with open(os.path.join ("테스트2.jpg"), "rb") as image_contents:
    results = predictor.classify_image(
        project.id, publish_iteration_name, image_contents.read())

    # Display the results.
    for prediction in results.predictions:
        print("\t" + prediction.tag_name +
              ": {0:.2f}%".format(prediction.probability * 100))